***
# ChatGPT API
***

In [ ]:
#!pip install gpt4all
#!pip install gpt4all langchain==0.0.342


In [ ]:
API_KEY = 'api key'


In [ ]:
import openai

openai.api_key  = API_KEY

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [ ]:
text = f"""
One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed , \
a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan .
"""

prompt = f"""
You will simplify this text in 6 different levels for the dyslexia patience\
re-write those instructions in the following format:

level 1 - ...
level 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text:")
print(response)


In [ ]:
text_2 = f"""
When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.
The floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.
At each end of the room, on the wall, hung a beautiful bear-skin rug.
These rugs were for prizes, one for the girls and one for the boys. And this was the game.
The girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.
This would have been an easy matter, but each traveller was obliged to wear snowshoes.
"""
prompt = f"""
You will simplify the text in 3 different difficulty level for dyslexia people:
0 is not dyslexia
1 is  dyslexia

re-write those instructions in the following format:

Level 1- ...
Level 2 - …
…
Level N - …

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)


In [ ]:
text_2 = f"""
When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.
The floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.
At each end of the room, on the wall, hung a beautiful bear-skin rug.
These rugs were for prizes, one for the girls and one for the boys. And this was the game.
The girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.
This would have been an easy matter, but each traveller was obliged to wear snowshoes.
"""
prompt = f"""
You will simplify the text in 4 different difficulty levels  :
Level 1 : Child
Level 2 : Teen
Level 3 : College students
Level 4 : Grad Students
Level 5 : Expert

re-write those instructions in the following format:

Level 1- ...
Level 2 - …
…
Level N - …


\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)


In [ ]:
text_2 = f"""
Hans Holbein the Younger acted as a matchmaker to Henry VIII,
tasked with painting the portraits of potential wives across Europe in a bid to safeguard the future of the throne. \
As two new exhibitions open, Deborah Nicholls-Lee looks at how he was the pre-eminent image-maker of the 16th Century.\
"""
prompt = f"""
You will simplify the text in 4 different difficulty levels for people who have dyslexia :
Level 1 : Child
Level 2 : Teen
Level 3 : College students
Level 4 : Grad Students
Level 5 : Expert


re-write those instructions in the following format:

Level 1- ...
Level 2 - …
…
Level N - …


\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)


In [ ]:
text_2 = f"""
Hans Holbein the Younger acted as a matchmaker to Henry VIII,
tasked with painting the portraits of potential wives across Europe in a bid to safeguard the future of the throne. \
As two new exhibitions open, Deborah Nicholls-Lee looks at how he was the pre-eminent image-maker of the 16th Century.\
"""
prompt = f"""

you will based on read ability level from pervious model to simplified the text_2:

the criteria is the following:

normal text : one side of the armed conflicts is composed mainly of the sudanese military and the janjaweed , a sudanese militia group recruited mostly from the afro-arab abbala tribes of the northern rizeigat region in sudan .
jeddah is the principal gateway to mecca , islam 's holiest city , which able-bodied muslims are required to visit at least once in their lifetime .
the great dark spot is thought to represent a hole in the methane cloud deck of neptune .
his next work , saturday , follows an especially eventful day in the life of a successful neurosurgeon .
the tarantula , the trickster character , spun a black cord and , attaching it to the ball , crawled away fast to the east , pulling on the cord with all his strength .
there he died six weeks later , on 13 january 888.

simplified text : who is involved in the conflict one side of the conflict is composed mainly of the janjaweed , a militia group recruited from the arab tribes who move from place to place herding camels .
jeddah is the main gateway to mecca , the holiest city of islam , where able-bodied muslims must go to at least once in a lifetime .
the great dark spot is thought to be a hole in the methane cloud deck of neptune .
his latest novel , saturday , follows an especially eventful day in the life of a neurosurgeon .
the tarantula , who knew what to do , spun a black cord and , attaching it to the ball , crawled away fast to the east , pulling on the cord with all his strength .
he died a few weeks later in january 888.

simplified text level 0 : one side of the armed conflicts consist of the sudanese military and the sudanese militia group janjaweed .
jeddah is the main gateway to mecca , islam 's holiest city , which able-bodied muslims are supposed to viisit at least once in their lifetime .
the great dark spot is thought to show a hole in the methane cloud deck of neptune .
his next work is saturday which is an eventful day in the life of a neurosuregon .
the tricky tarantula spun a black web and attached it to the ball . afterwards , it crawled away and pulled the web with him .
he died there six weeks later on january 13 , 888.

simplified text level 1 : one side of the armed conflicts is mainly sudanese military and the janjaweed , which recruited from the afro-arab abbala tribes .
jeddah is the main entrance to mecca , islam 's holiest city , which pure muslims are required to visit at least once in their lifetime .
the great dark spot is thought to be a hole in the methane cloud deck of neptune .
his next work , saturday , follows a fun day in the life of a good doctor .
the tarantula spun a black cord and crawled away fast to the east while pulling on the cord with its strength .
here he died six weeks later , on 13 january 888.

simplified text level 2 : one side of the armed conflicts is composed mainly of the sudanese military and the janjaweed , a sudanese militia group recruited mostly from the afro-arab abbala tribes in sudan .
jeddah is the gateway to mecca which is islam 's holiest city and muslims are required to visit at least once .
the great dark spot is thought to represent a hole in the methane .
his next work , saturday , will show how very busy and varying the life of a brain doctor is .
the trickster spun a black cord and , attaching it to the ball , crawled away fast to the east , pulling on the cord with all his strength .
he died six weeks later on january 13th 888.


you will based on the citeria provided above to generate the simplified text for dyslexia people:

Level 1 : simplified text level 2
Level 2 : simplified text level 1
Level 3 : simplified text level 0
Level 4 : normal text


re-write those instructions in the following format:

Level 1- ...
Level 2 - …
…
Level N - …


\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)


***
# LangChain
***

# Imports

In [1]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate

# Use GPT4ALL

In [2]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])


In [8]:
local_path=("/Users/christine/Downloads/mistral-7b-openorca.Q4_0.gguf")

In [9]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)


In [10]:
# create LLM chain
llm_chain = LLMChain(prompt=prompt, llm=llm)


In [11]:
# request 1
question1 = """can you simplify this text for dyslexia people:
When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.
The floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.
At each end of the room, on the wall, hung a beautiful bear-skin rug.
These rugs were for prizes, one for the girls and one for the boys. And this was the game.
The girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.
This would have been an easy matter, but each traveller was obliged to wear snowshoes."""

llm_chain.run(question1)


 The young people came back into the ballroom and saw it had changed. It looked like a winter scene with lots of snow on the floor. There were many palms and evergreens, covered in fake snow (flour) and cotton to look like real snow. They even added diamond dust and crystal icicles for decoration. At each end of the room there was a bear-skin rug which would be given as prizes - one for girls and one for boys. The game was to reach your pole, either North or South Pole, while wearing snowshoes.

' The young people came back into the ballroom and saw it had changed. It looked like a winter scene with lots of snow on the floor. There were many palms and evergreens, covered in fake snow (flour) and cotton to look like real snow. They even added diamond dust and crystal icicles for decoration. At each end of the room there was a bear-skin rug which would be given as prizes - one for girls and one for boys. The game was to reach your pole, either North or South Pole, while wearing snowshoes.'

In [7]:
# request 2
question2 = """can you simplify this text for dyslexia people:
Owls are the order Strigiformes, comprising 200 bird of prey species.
Owls hunt mostly small mammals, insects, and other birds though some species specialize in hunting fish."""

llm_chain.run(question2)


Sure, I can simplify the text for dyslexia people. Here is a simplified version of the text:

Owls are birds that belong to the order Strigiformes. They eat small mammals, insects, and other birds. Some owls specialize in hunting fish.

'\nSure, I can simplify the text for dyslexia people. Here is a simplified version of the text:\n\nOwls are birds that belong to the order Strigiformes. They eat small mammals, insects, and other birds. Some owls specialize in hunting fish.'

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain.document_loaders import WikipediaLoader

# Load content from Wikipedia using WikipediaLoader
loader = WikipediaLoader("Machine_learning")
docs= loader.load()

#split in chunks of 2000 characters - max input size for GPT 2000 and a bit
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(docs)

#get embeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

persist_directory = 'db/chroma_3/'

# Create the vector store
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)


In [ ]:
template = """Please generate a short summary of machine learning
for the dyslexic people.


Context: {context}

Human: {question}
Assistant:"""

prompt = PromptTemplate(
input_variables=["context",  "question"], template=template)


In [ ]:
#retrieval
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    verbose=False,
    chain_type_kwargs={
        "verbose": False,
        "prompt": prompt
    }
)


In [ ]:
res = qa("generate a 100 words simplifiy text of Machine_learning for dyslexic students")


# Prompting

In [16]:
from langchain.llms import GPT4All
model_path = "/Users/christine/Downloads/gpt4all-falcon-q4_0.gguf"
llm = GPT4All(model=model_path, verbose=False)

In [17]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

prompt_template = " please simplify the following text for dyslexic patients :\
Mr. Grimes was to come up next morning to Sir John Harthover's, \
at the Place, for his old chimney-sweep was gone to prison, \
and the chimneys wanted sweeping. And so he rode away, \
not giving Tom time to ask what the sweep had gone to prison for, \
which was a matter of interest to Tom, as he had been in prison once or twice himself. \
Moreover, the groom looked so very neat and clean, with his drab gaiters, drab breeches, \
drab jacket, snow-white tie with a smart pin in it, and clean round ruddy face, that Tom was offended and disgusted at his appearance, and considered him a stuck-up fellow, who gave himself airs because he wore smart clothes, and other people paid for them; and went behind the wall to fetch the half-brick after all; but did not, remembering that he had come in the way of business, and was, as it were, under a flag of truce."

llm = GPT4All(model=model_path, verbose=False)
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain.predict()

"\nMr. Grimes was supposed to visit Sir John Harthover's house the next morning for his old chimney sweep had gone to prison and the chimneys needed sweeping. Tom was curious about why the chimney sweep went to prison but did not have time to ask before Mr. Grimes rode away. The groom, who looked very neat and clean, with his drab gaiters, drab breeches, drab jacket, snow-white tie with a smart pin in it, and clean round ruddy face, made Tom feel offended and disgusted at his appearance. He considered the groom a stuck-up fellow who gave himself airs because he wore smart clothes, and other people paid for them. Tom went behind the wall to fetch the half-brick after all, but did not remember that he had come in the way of business, and was, as it were, under a flag of truce."

# PDF reader

In [71]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/Users/christine/Downloads/test.pdf")
documents = loader.load_and_split()

In [72]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
text = text_splitter.split_documents(documents)

In [73]:
print(text)

[Document(page_content='1START  ENTREPRENEURIAL MANAGEMENT Building a startup is an exercise in ins>tu>on building; thus, it necessarily involves management. This oHen comes as a surprise to aspiring entrepreneurs, because their associa>ons with these two words are so diametrically opposed. Entrepreneurs are rightly wary of implemen>ng tradi>onal management prac>ces early on in a startup, afraid that they will invite bureaucracy or s>ﬂe crea>vity. Entrepreneurs have been trying to ﬁt the square peg of their unique problems into the round hole of general management for decades. As a result, many entrepreneurs take a “just do it” aRtude, avoiding all forms of management, process, and discipline. Unfortunately, this approach leads to chaos more oHen than it does to success. I should know: my ﬁrst startup failures were all of this kind. The tremendous success of general management over the last century has provided unprecedented material abundance, but those management principles are ill s

In [74]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [75]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(text, embeddings, persist_directory="db")

In [76]:
from langchain.llms import GPT4All
model_path = "/Users/christine/Downloads/gpt4all-falcon-q4_0.gguf"
llm = GPT4All(model=model_path, verbose=False)

In [77]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

In [78]:
res = qa('''
        Can you reprint the provided PDF for me in concise paragraphs in a dyslexia friendly writing style?
        ''')

In [79]:
simplification = res.get('result', None) if isinstance(res.get('result', None), str) else None
print(simplification)

 

Sure, here's a summary of the main points from the PDF:

* The Risk of Bias in Systematic Reviews (ROBINS-I) is a tool that helps reviewers assess the risk of bias in systematic reviews.
* The tool consists of a set of signaling questions that are designed to be used in conjunction with the Cochrane Handbook for Systematic Reviews and Meta-Analyses.
* The signaling questions are intended to help reviewers identify potential sources of bias in the studies they are reviewing, and to assess the overall risk of bias in the systematic review as a whole.
* The tool is based on the GRADE (Grading of Recommendations, Assessment, Development and Evaluation) approach, which provides a framework for assessing the quality of evidence from systematic reviews.
* The tool has been piloted and tested by several working groups, including the Cochrane Collaboration, and has been applied in small groups to six papers.
* The tool is intended to be used in conjunction with the Cochrane Handbook for Syst